# Chapter 2 - Tokens and Embeddings

In [1]:
# %%capture
!pip install transformers>=4.41.2 sentence-transformers>=3.0.1 gensim>=4.3.2 scikit-learn>=1.5.0 accelerate>=0.31.0

## 2.1 下载一个开源模型运行看看

和前一章一样，分开加载并且分别解释不同部分的作用

In [2]:
## step1: 加载模型

from transformers import AutoModelForCausalLM, AutoTokenizer

# 加载模型和 tokenizer 
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

>  解释原文中为什么要写一个 <|assistant|>？这是因为原文中用得是：microsoft/Phi-3-mini-4k-instruct，它用 <|assistant|> 表示 AI 回答开始。构成的格式如下：
> ```
> <|system|>
> You are a helpful assistant.<|end|>
> <|user|>
> How to explain Internet for a medieval knight?<|end|>
> <|assistant|>
> ```


```python

# 原文。这里的 <|assistant|> 是为了让模型能
prompt = "帮我写一个请教条，原因是自己生病了。<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
```

Qwen 的模型和它有什么区别呢？
```
<|im_start|>user
讲一个猫有关的笑话？<|im_end|>
<|im_start|>assistant
```


In [3]:
## 所以对于 Qwen 可以这么做？ 
### 中文 
prompt = "帮我写一个请教条，原因是自己生病了。<|im_start|>assistant"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


帮我写一个请教条，原因是自己生病了。<|im_start|>assistant: 尊敬的医生，您好！我最近感觉身体不适，想咨询一下您的意见


In [4]:
print(input_ids)

tensor([[108965,  61443,  46944, 116069,  38989,   3837, 107711,  99283, 109281,
          34187,   1773, 151644,  77091]], device='cuda:0')


In [5]:
for id_ in input_ids[0]:
    print(tokenizer.decode(id_))

帮我
写
一个
请教
条
，
原因是
自己
生病
了
。
<|im_start|>
assistant


In [7]:
input_ids.size()

torch.Size([1, 13])

In [8]:
## 英文
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Dear Sarah,

I hope this message finds you well. I am writing to apologize for the unfortunate incident


In [9]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

Write
 an
 email
 apolog
izing
 to
 Sarah
 for
 the
 tragic
 gardening
 mish
ap
.
 Explain
 how
 it
 happened
.<
|
assistant
|
>


## 2.2 对比训练过的 LLM tokenizers

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    '''
    这个函数首先加载指定的分词器，然后对输入的句子进行编码，得到token的ID列表。
    接着，它遍历这些token ID，使用对应的颜色高亮显示每个token，并在控制台中打印出来。
    颜色是通过ANSI转义序列设置的，其中\x1b[0;30;48;2;{color}m是设置背景色的转义序列，\x1b[0m是重置所有属性的转义序列。
    '''
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [11]:
show_tokens(text, "bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [12]:
show_tokens(text, "gpt2")


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [13]:
show_tokens(text, "Qwen/Qwen2.5-0.5B-Instruct")


 English  and  CAPITAL IZATION 
 🎵  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 1 2 . 0 * 5 0 = 6 0 0 
 

| 模型                        | 分词算法                               | 词表特点                            | 对同一文本的分词差异来源                                                                     |
| ------------------------- | ---------------------------------- | ------------------------------- | -------------------------------------------------------------------------------- |
| **bert-base-uncased**     | WordPiece                          | 子词级，30 k 左右，大小写不敏感，无中文、无 emoji  | 1. 把大写转成小写<br>2. 遇到未登录词（中文、emoji、符号）用 `[UNK]` 或拆成极碎子词<br>3. 用 `##` 标记“继续子词”      |
| **gpt2**                  | BPE（Byte-level BPE）                | 50 k 左右，**字节级**，保留大小写，无 `[UNK]` | 1. 任何字节序列都能表示，\*\*不会出现 `[UNK]`<br>2. 中文、emoji、符号会被拆成字节对，看起来“更碎”<br>3. 空格用 `Ġ` 标记 |
| **Qwen2.5-0.5B-Instruct** | **BBPE**（Byte-level BPE 升级版）+ 中文词表 | 151 k，**中英混合词表**，保留大小写          | 1. 中文整词或高频子词直接进词表，**颗粒度最粗**<br>2. 英文、数字、符号与 GPT-2 类似，但词表更大，碎的程度更低                |


## 2.3 理解 Embedding

### 2.3.1 原始 word2vec embedding （和 LLM 无关）

In [15]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

model.most_similar([model['king']], topn=11)

[=================---------------------------------] 35.5% 23.4/66.0MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[========================================----------] 82.0% 54.1/66.0MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 66.0/66.0MB downloaded


[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]

### 2.3.2 上下文相关的 Embedding

Contextualized Word Embeddings From a Language Model (Like BERT)，最早应该是 ElMO， 然后是 bert

传统词向量（如 Word2Vec）是静态的：每个词只有一个固定向量。而 BERT 这样的语言模型可以根据上下文动态生成词向量。例如：

- “bank” 在 “river bank” 和 “money bank” 中会有不同的向量。

这就是所谓的 上下文相关性。

In [16]:
from transformers import AutoModel, AutoTokenizer

# 加载 BERT 模型和分词器  AutoTokenizer 会将句子转成 BERT 能理解的 token 格式
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load a language model 
model = AutoModel.from_pretrained("bert-base-uncased")

# 对句子进行编码
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [17]:
output # input_ids 是词的编号

tensor([[[-0.1689,  0.1361, -0.1394,  ..., -0.6251,  0.0522,  0.3671],
         [-0.3633,  0.1412,  0.8800,  ...,  0.1043,  0.2888,  0.3727],
         [-0.6986, -0.6988,  0.0645,  ..., -0.2210,  0.0099, -0.5940],
         [ 0.8310,  0.1237, -0.1512,  ...,  0.1031, -0.6779, -0.2629]]],
       grad_fn=<NativeLayerNormBackward0>)

In [20]:
output.shape

torch.Size([1, 4, 768])

In [21]:
# [CLS] 是句首标记，[SEP] 是句尾标记
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
hello
world
[SEP]


### 2.3.3 文本向量（句向量）
Text Embeddings (For Sentences and Whole Documents)

In [22]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

# Convert text to text embeddings
vector = model.encode("测试一个小模型的 embedding 能力")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
vector.shape

(768,)

### 2.3.4 基于 embeddings的歌曲推理

In [28]:
# !pip install pandas

In [33]:
import pandas as pd
from urllib import request

# 第一步：下载播放列表数据
# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

In [36]:
lines[0]

'0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 2 42 43 44 45 46 47 48 20 49 8 50 51 52 53 54 55 56 57 25 58 59 60 61 62 3 63 64 65 66 46 47 67 2 48 68 69 70 57 50 71 72 53 73 25 74 59 20 46 75 76 77 59 20 43 '

In [37]:
# 第二步：过滤掉只有一首歌的播放列表
# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

In [40]:
# 第三步：下载歌曲元数据
# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
# 每行格式是：歌曲ID\t标题\t艺术家
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [41]:
songs_df

,title,artist
id,,
0,Gucci Time (w\/ Swizz Beatz),Gucci Mane
1,Aston Martin Music (w\/ Drake & Chrisette Mich...,Rick Ross
2,Get Back Up (w\/ Chris Brown),T.I.
3,Hot Toddy (w\/ Jay-Z & Ester Dean),Usher
4,Whip My Hair,Willow
...,...,...
75258,USA Today,Alan Jackson
75259,Superstar,Raul Malo
75260,Romancin' The Blues,Giacomo Gates


In [30]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [42]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, # 一个列表，每个元素是一个播放列表（由歌曲 ID 构成）
    vector_size=32,
    window=20,
    negative=50, 
    min_count=1, 
    workers=4
) # Word2Vec 把每个播放列表当作一个“句子”，每个歌曲 ID 当作一个“词” 模型学习哪些歌曲经常一起出现，从而构建一个向量空间

In [43]:
song_id = 2172
# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id)) # 相似性是通过 余弦相似度（cosine similarity） 计算的：

[('2004', 0.9974495768547058),
 ('2886', 0.9973307847976685),
 ('2715', 0.9971328973770142),
 ('2688', 0.9970644116401672),
 ('2976', 0.9970182180404663),
 ('6706', 0.996772825717926),
 ('2867', 0.9967395663261414),
 ('10055', 0.9966088533401489),
 ('2825', 0.9965750575065613),
 ('2810', 0.9965003728866577)]

In [51]:
songs_df.iloc[[2172,2004,2886,2715,2688]]

,title,artist
id,,
2172,Fade To Black,Metallica
2004,For Those About To Rock (We Salute You),AC\/DC
2886,The Zoo,Scorpions
2715,Rainbow In The Dark,Dio
2688,Holy Diver,Dio


In [53]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
2004,For Those About To Rock (We Salute You),AC\/DC
2886,The Zoo,Scorpions
2715,Rainbow In The Dark,Dio
2688,Holy Diver,Dio
2976,I Don't Know,Ozzy Osbourne


In [54]:
print(songs_df.iloc[842])

print_recommendations(842)

title     California Love (w\/ Dr. Dre & Roger Troutman)
artist                                              2Pac
Name: 842 , dtype: object


,title,artist
id,,
890,Knock You Down (w\/ Ne-Yo & Kanye West),Keri Hilson
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
330,Hate It Or Love It (w\/ 50 Cent),The Game
838,Imma Be,Black Eyed Peas
893,Whatever You Like,T.I.


为什么这种方法有效？

因为在真实用户行为中： 如果歌曲 A 和 B 经常一起出现在播放列表中，说明它们可能风格相近、受众相似

Word2Vec 能捕捉这种“共现”关系，从而在向量空间中把它们拉近